In [1]:
import tensorflow as tf
tf.executing_eagerly()

2023-07-27 14:21:45.458426: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-27 14:21:45.498087: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-27 14:21:45.499566: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-27 14:21:46.318550: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


True

In [2]:
x = [[2,]]
x

[[2]]

In [3]:
m = tf.matmul(x, x)  # 矩阵矩阵相乘
print(m)

tf.Tensor([[4]], shape=(1, 1), dtype=int32)


In [4]:
m.numpy()

array([[4]], dtype=int32)

In [5]:
a = tf.constant([[1, 2],
                [3, 4]])
a

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4]], dtype=int32)>

In [6]:
a.numpy()

array([[1, 2],
       [3, 4]], dtype=int32)

In [7]:
b = tf.add(a, 1)  # 矩阵加法
b

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[2, 3],
       [4, 5]], dtype=int32)>

In [8]:
c = tf.multiply(a, b)  # 矩阵相乘
c

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 2,  6],
       [12, 20]], dtype=int32)>

In [9]:
num = tf.convert_to_tensor(10)
num

<tf.Tensor: shape=(), dtype=int32, numpy=10>

In [10]:
for i in range(num.numpy()):
    i = tf.constant(i)  # 建立tensor常亮
    if int(i % 2) == 0:
        print("even")
    else:
        print('odd')

even
odd
even
odd
even
odd
even
odd
even
odd


In [11]:
import numpy as np

In [12]:
d = np.array([[5, 6], [7, 8]])
d

array([[5, 6],
       [7, 8]])

In [13]:
(a+b).numpy()

array([[3, 5],
       [7, 9]], dtype=int32)

In [14]:
g = tf.convert_to_tensor(10)
g

<tf.Tensor: shape=(), dtype=int32, numpy=10>

In [15]:
float(g)  # 将tensor类型转化为python类型

10.0

# 变量与自动微分运算

In [16]:
import tensorflow as tf

In [17]:
v = tf.Variable(0.0)  # 创建变量
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.0>

In [18]:
v+1

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [19]:
v.assign(5)  # 直接更改变量
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.0>

In [20]:
v.assign_add(1)  # 加1
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=6.0>

In [21]:
v.read_value()  # 读取变量
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=6.0>

### 变量求微分

In [22]:
w = tf.Variable([[1.0]])
w

<tf.Variable 'Variable:0' shape=(1, 1) dtype=float32, numpy=array([[1.]], dtype=float32)>

In [23]:
with tf.GradientTape() as t:
    loss = w*w

In [24]:
grad = t.gradient(loss, w)  # 求解loss对w的微分
grad

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[2.]], dtype=float32)>

### 常量求微分

In [25]:
w = tf.constant(3.0)
with tf.GradientTape() as t:
    t.watch(w)
    loss = w*w

In [26]:
dloss_dw = t.gradient(loss, w)  # gradient函数只能计算一次，计算过后释放资源。

In [27]:
dloss_dw

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

连续求微分

In [28]:
w = tf.constant(3.0)
with tf.GradientTape(persistent=True) as t:
    t.watch(w)
    y = w*w
    z = y*y

In [29]:
dy_dw = t.gradient(y, w)
dy_dw

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [30]:
dz_dz = t.gradient(z, w)  # 这里会出现错误，因为gradient已经被释放
dz_dz

<tf.Tensor: shape=(), dtype=float32, numpy=108.0>

# Mnist 数据集

In [101]:
(train_image, train_labels), (test_image,test_labels) = tf.keras.datasets.mnist.load_data()

In [79]:
train_image.shape

(60000, 28, 28)

In [102]:
# 扩增维度
train_image = tf.expand_dims(train_image, -1)  
test_image=tf.expand_dims(test_image,-1)
train_image.shape,test_image.shape

(TensorShape([60000, 28, 28, 1]), TensorShape([10000, 28, 28, 1]))

In [81]:
# 归一化，更改数据类型-》float32
train_image = tf.cast(train_image/255, tf.float32)  
test_image = tf.cast(test_image/255, tf.float32)
# label类型转换
train_labels = tf.cast(train_labels, tf.int64)  
test_labels = tf.cast(test_labels, tf.int64)

In [82]:
# 通过元组的方式建立数据集
dataset = tf.data.Dataset.from_tensor_slices(
    (train_image, train_labels)
)

test_dataset= tf.data.Dataset.from_tensor_slices(
    (test_image, test_labels)
)

In [83]:
dataset = dataset.shuffle(300)  # 乱序
dataset = dataset.batch(32)  # 批次
test_dataset = test_dataset.batch(32)  # 批次

In [99]:
test_dataset,dataset

(<_BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <_BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

In [84]:
#定义模型
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(
        16, [3, 3], activation='relu', input_shape=(None, None, 1)),
    tf.keras.layers.Conv2D(32, [3, 3], activation='relu'),
    tf.keras.layers.GlobalMaxPool2D(),  # 全局平均池化
    tf.keras.layers.Dense(10)
])

---

In [85]:
#model.trainable_variables  # 可训练参数

In [86]:
# 定义优化函数
optimizer = tf.keras.optimizers.Adam()  
# 定义损失函数
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True)  

In [87]:
features, labels = (next(iter(dataset)))  # 迭代器
features.shape, labels.shape

(TensorShape([32, 28, 28, 1]), TensorShape([32]))

In [88]:
predictions = model(features)
predictions.shape  # 这里32的原因是，定义的一个批次是32，因此每一批次中有32个数据

TensorShape([32, 10])

In [89]:
tf.argmax(predictions, axis=1)  # 查看数据输入模型后的输出值

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([8, 2, 8, 2, 8, 2, 2, 8, 8, 2, 2, 2, 8, 2, 2, 2, 8, 8, 2, 2, 8, 8,
       2, 8, 2, 8, 2, 2, 8, 8, 2, 2])>

In [90]:
labels

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([1, 8, 9, 7, 1, 5, 3, 2, 4, 9, 4, 6, 6, 5, 8, 7, 4, 1, 8, 5, 1, 4,
       7, 8, 9, 9, 1, 7, 4, 8, 7, 0])>

---

In [91]:
def loss(model, x, y):
    y_ = model(x)  # y_是经过模型后预测的值，y是真值
    return loss_func(y, y_)  # 返回损失值

In [92]:
#定义汇总计算对象
train_loss=tf.keras.metrics.Mean('train_loss')
train_accuracy=tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')

test_loss=tf.keras.metrics.Mean('test_loss')
test_accuracy=tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [93]:
# 训练一步
def train_step(model, images, labels):
    with tf.GradientTape() as t:  # 追踪损失函数的计算
        pred=model(images)#计算预测结果
        loss_step=loss_func(labels,pred)
        
    grads = t.gradient(loss_step, model.trainable_variables)
    # 通过优化函数，改变train_variables变量值，使梯度下降最快
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    train_loss(loss_step)#计算损失值
    train_accuracy(labels,pred)#计算准确率

def test_step(model, images, labels):
    pred=model(images)#计算预测结果
    loss_step=loss_func(labels,pred)
    
    test_loss(loss_step)#计算损失值
    test_accuracy(labels,pred)#计算准确率

In [96]:
def train():
    for epoch in range(10):
        print("")
        print("epoch:{}".format(epoch))
#         for (batch, (images, balels)) in enumerate(dataset):
#             train_step(model, images, labels)  # 进行一步训练
#             print("\rstep:{}, loss:{:.4f}, acc:{:.4f}".format(batch,
#                                                               train_loss.result(),
#                                                               train_accuracy.result()), end="")

        for (batch, (images, balels)) in enumerate(test_dataset):
            test_step(model, images, labels)  # 进行一步训练
            print("\rstep:{}, test_loss:{:.4f}, test_acc:{:.4f}".format(batch,
                                                              test_loss.result(),
                                                              test_accuracy.result()), end="")
            # 重置状态
        train_loss.reset_states()
        train_accuracy.reset_states()

        test_loss.reset_states()
        test_accuracy.reset_states()

In [97]:
train()


epoch:0
step:311, test_loss:2.1414, test_acc:0.1512

ValueError: `labels.shape` must equal `logits.shape` except for the last dimension. Received: labels.shape=(32,) and logits.shape=(16, 10)

# tf.keras.metrics 汇总计算模块

In [63]:
m = tf.keras.metrics.Mean('acc')

In [64]:
m(10), m(20)

(<tf.Tensor: shape=(), dtype=float32, numpy=10.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=15.0>)

In [65]:
m.result()  # 返回均值

<tf.Tensor: shape=(), dtype=float32, numpy=15.0>

In [66]:
m([30, 40])

<tf.Tensor: shape=(), dtype=float32, numpy=25.0>

In [67]:
m.result()

<tf.Tensor: shape=(), dtype=float32, numpy=25.0>

In [68]:
m.reset_states()  # 数据初始化

In [71]:
m(1)

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [72]:
a = tf.keras.metrics.SparseCategoricalAccuracy('acc')  # 初始化计算准确率对象
a(labels, model(features))  # 计算正确率

<tf.Tensor: shape=(), dtype=float32, numpy=0.15625>

In [73]:
5/32  # 每一批次32个数据，根据model(feature)计算出，只有4个正确，因此准确率为5/32
# 准确率与sparsecategoricalaccuracy计算无误

0.15625